### 회귀(Regression)

- 주어진 독립변수(피처)와 종속변수(결정 값) 데이터 기반에서 학습을 통해 최적의 회귀 계수를 찾는 것
    - 회귀는 여러 개의 독립변수와 한 개의 종속변수 간의 상관관계를 모델링하는 기법

#### 회귀모델 성능지표

- MAE(Mean Absolute Error) : 실제 값과 예측값의 차이를 절대값으로 변환해 평균
- MSE(Mean Squared Error) : 실제 값과 예측값의 차이를 제곱해 평균
- RMSE(Root Mean Absolute Error) : MAE에 루트를 씌운 것, MSE 값은 실제값과 예측값의 차이를 제곱하여 평균한 것이기 때문에 실제 오차보다 커지는 현상이 발생
- R^2 : 
    - 1-(추정모형의 MSE/평균 관측 값의 MSE)로 계산
    - 0에서 1사이의 값
    - 선형 모델의 오차가 작다면 R squared가 커지게 되고 선형 모델의 오차가 크다면 R squared가 작아지게 됨

#### LinearRegression을 이용한 보스턴 주택 가격 예측

- CRIM : 지역별 범죄 발생률
- ZN : 25,000평방피트를 초과하는 거주 지역 비율
- INDUS : 비상업 지역 넓이 비율
- CHAS : 찰스강에 대한 더미변수(강의 경계에 위치한 경우 1, 아니면 0)
- NOX : 일산화질소 농도
- RM : 거주할 수 있는 방 개수
- AGE : 1940년 이전에 건축된 소유 주택의 비율
- DIS : 5개 주요 고용센터까지의 가중 거리
- RAD : 고속도로 접근 용이도
- TAX : 10,000 달러 당 재산세율
- PTRATIO : 지역의 교사와 학생 수 비율
- B : 지역의 흑인 거주 비율
- LSTAT : 하위 계층의 비율
- PRICE(target) : 본인 소유의 주택 가격(중앙값)